# NeuralNetworkにおける最適化手法

## 学習方法

### バッチ学習

### オンライン学習

### ミニバッチ学習

## 最適化アルゴリズム

### 最急降下法

#### 更新則

$$
\theta_{t} := \theta_{t-1} - \eta\nabla J(\theta_{t-1})
$$

#### パラメーター

|パラメーター|説明|推奨値|
|:-----:|:-----:|:-----:|
|$\eta$|学習率|?|

#### 説明

標準的な最適化アルゴリズムである。

### モメンタム

#### 更新則

$$
\begin{aligned}
\nu_{t} &= \gamma \nu_{t-1} + \eta \nabla J(\theta_{t-1})\\
\theta_{t} &:= \theta_{t-1} - \nu_{t}
\end{aligned}
$$

なお、上記の更新規則は、次のように書き換えることも可能である。
(ここで、$\Delta_{t} = \theta_{t} - \theta_{t-1}$とする。)

$$
\theta_{t} := \theta_{t-1} + \gamma \Delta_{t-1} - \eta \nabla J(\theta_{t-1})
$$

(証明)  
更新式の第1式より、
$$
\nu_{t} = -\Delta_{t}
$$

となる。上式を更新式の第1式に代入し、その結果を第2式に代入すると、
$$
\theta_{t} := \theta_{t-1} - (-\gamma \Delta_{t-1} + \eta \nabla J(\theta_{t-1}) = \theta_{t-1} + \gamma\Delta_{t-1} - \eta \nabla J(\theta_{t-1})
$$


#### パラメーター

|パラメーター|説明|推奨値|
|:-----:|:-----|:-----|
|$\eta$|学習率|``0.1``?|
|$\gamma$|モメンタム|``0.9``|

#### 説明

前回のパラメーター更新の方向に対する慣性を持つ。無駄に振動することがなくなり(勾配方向に加速し)、効率的な収束が期待できる。

### NAG

#### 更新則

$$
\begin{aligned}
\nu_{t} &= \gamma \nu_{t-1} + \eta\nabla J(\theta_{t-1} - \gamma \nu_{t-1})\\
\theta_{t} &:= \theta_{t-1} - \nu_{t}
\end{aligned}
$$

#### パラメーター

モメンタムと同様。

#### 説明

NAG(Nesterov accelerated gradient)は、モメンタムをやや改良している。
具体的には、勾配の計算に$\theta_{t-1} - \gamma \nu_{t-1}$を使用している。  
これは、$t$回目の更新後の$\theta_{t}$の勾配の近似値を計算していることとなる。

### AdaGrad

#### 更新則

まず、ステップ$t$におけるパラメーター$\theta_{i}$に関する目的関数の勾配を$g_{t}$とし、次のように表記する。
$$
g_{t} = \nabla J(\theta)
$$

次に、ステップ$t$までの勾配の累積を次の更新則で求める。(ここで、$\circ$はアダマール積)
$$
\begin{aligned}
G_{0} &= \epsilon\\
G_{t} &= G_{t-1} + g_{t}\circ g_{t}
\end{aligned}
$$

Adagradの更新規則は、上記の勾配を次のように更新する。

$$
\theta_{t} := \theta_{t - 1} - \frac{\eta}{\sqrt{G_{t}}}\circ g_{t}
$$

すなわち、各ステップで求めた勾配の各要素の二乗の累積を分母とした係数を用いて、各パラメーターを更新していく。  
なお、$\epsilon$はゼロ除算を避ける平滑化項であり、大抵は$1e-8$のオーダーである。

#### パラメーター

|パラメーター|説明|推奨値|
|:-----:|:-----:|:-----:|
|$\eta$|学習率|``0.001``?|
|$\epsilon$|平滑化項|$1e-8$?|
|$G_0$|勾配の累積二乗和|$\epsilon$|

#### 説明

AdaGradは、各パラメーター$\theta_i$毎に異なる学習率を使用する。  
勾配に乗じる分母に、各反復で求めた勾配の二乗を累積していくため、学習率は反復回数に応じて減少していく。

### AdaDelta

#### 更新則

AdaGradのように勾配の二乗を累積させる代わりに、次のように勾配の二乗の期待値(移動平均)を定義する。
$$
E[g^{2}]_{t} = \gamma E[g^{2}]_{t-1} + (1 - \gamma)g^{2}_t
$$

また、パラメーターの変化(差分)の二乗の期待値を(移動平均)を次のように定義する。
$$
E[\Delta\theta^{2}]_{t} = \gamma E[\Delta\theta^{2}]_{t-1} + (1 - \gamma)\Delta\theta^{2}_{t}
$$

これらを、$G_t$および$\eta$に代入したものが、AdaDeltaである。

$$
\theta_{t} := \theta_{t-1} - \frac{\sqrt{E[\theta^{2}]_{t-1} + \epsilon}}{\sqrt{E[g^{2}]_{t} + \epsilon}}\circ g_{t}
$$

なお、$\Delta\theta_{t}$はまさに更新式の第二項であり、更新時にはこの項を用いて上記アルゴリズムを計算すれば良い。

#### パラメーター

|パラメーター|説明|推奨値|
|:-----:|:-----:|:------:|
|$\gamma$|移動平均の重み|``0.95``?``0.9``?|
|$E[g^{2}]_{0}$|勾配の二乗の期待値|``0``|
|$E[\theta^{2}]_{0}$|パラメーター変化の二乗の期待値|``0``|

#### 説明

AdaDeltaは、急速かつ単調な学習率の低下を防ぐためにAdaGradを改良したもの。  
学習率を初期値に与える必要がない。

なお、$\sqrt{E[g^{2}]_{t} + \epsilon}$は勾配の$RMS$、$\sqrt{E[\theta^{2}]_{t} + \epsilon}$はパラメーター差分の$RMS$であることから、次の式のようにも表現できる。

$$
\theta_{t} := \theta_{t - 1} - \frac{RMS[\theta]_{t-1}}{RMS[g]_{t}}\circ g_{t}
$$

### RMSProp

#### 更新則

AdaDeltaと同様、勾配の二乗の期待値を次のように定義する。

$$
E[g^{2}]_{t} = \gamma E[g^{2}]_{t-1} + (1 - \gamma)\circ g^{2}_{t}
$$

更新式は次の通り。

$$
\theta_{t} := \theta_{t-1} - \frac{\eta}{\sqrt{E[g^{2}]_{t} + \epsilon}}\circ g_{t}
$$

#### パラメーター

|パラメーター|説明|推奨値|
|:-----:|:-----:|:-----:|
|$\eta$|学習率|``0.001``|
|$\gamma$|移動平均の重み|``0.9``|

#### 説明

AdaGradの改良版であり、AdaDeltaを若干シンプルにしたもの。  
Courseraの講義でHinton教授が提案した手法であり、$\eta$が``0.001``のときに$\gamma$は``0.9``にすることを推奨している。

### Adam

#### 更新則

Adamは、RMSPropで用いた勾配の二乗の期待値に加え、勾配の期待値を使用する。

$$
\left\{
\begin{aligned}
m_{t} = \beta_{1}m_{t-1} + (1 - \beta_{1})\circ g_{t}\\
v_{t} = \beta_{2}v_{t-1} + (1 - \beta_{2})\circ g^{2}_{t}
\end{aligned}
\right.
$$

求めた$m_{t}, v_{t}$のバイアスを補正する。

$$
\begin{aligned}
\hat{m}_{t} = \frac{m_{t}}{1 - \beta^{t}_{1}}\\
\hat{v}_{t} = \frac{v_{t}}{1 - \beta^{t}_{2}}
\end{aligned}
$$

これらを用いて、次式でパラメーターの更新を行う。
$$
\theta_{t} := \theta_{t-1} - \frac{\eta}{\sqrt{\hat{v}_{t}} + \epsilon}\circ \hat{m}_{t}
$$

なお、$t=1$からスタートする点に注意する。

#### パラメーター

|パラメーター|説明|推奨値|
|:-----:|:-----:|:-----:|
|$\beta_{1}$|勾配の移動平均の重み|``0.9``|
|$\beta_{2}$|勾配の二乗の移動平均の重み|``0.999``|
|$\eta$|学習率|``0.001``|
|$\epsilon$|平滑化項|``10e-8``|

#### 説明

AdagradやRMSpropに勾配の期待値を追加している。  
また、それらは勾配の一次・二次モーメントの概算値とみなすことができるため、バイアス補正を行う。
得られた値をもとに、パラメーターを更新していく。

### RMSPropGraves

#### 更新則

Adamと同様、勾配および勾配の二乗の移動平均を定義する。(ただし、重みは共有する)

$$
\begin{aligned}
n_{t} &= \gamma n_{t -1} + (1 - \gamma) \circ g^{2}_{t}\\
m_{t} &= \gamma m_{t-1} + (1 - \gamma) \circ g_{t}
\end{aligned}
$$

以上を用いて、次の更新式でパラメーターを更新する。

$$
\Delta_{t} = \mu \Delta_{t-1} - \frac{\eta}{\sqrt{n_{t} - m^{2}_{t} + \epsilon}}\circ g_{t}
$$

$$
\theta_{t} = \theta_{t-1} + \Delta_{t}
$$

#### パラメーター

|パラメーター|説明|推奨値|
|:-----:|:-----:|:-----:|
|$\gamma$|移動平均の重み|``0.95``|
|$\mu$|モメンタム|``0.9``|
|$\eta$|学習率|``0.0001``|
|$\epsilon$|平滑化項|``0.0001``|


#### 説明

### 参考

- [勾配降下法の最適化アルゴリズムを概観する](http://postd.cc/optimizing-gradient-descent/)
- [確率的勾配降下法 - Wikipedia](https://ja.wikipedia.org/wiki/確率的勾配降下法)
- 岡谷貴之:「深層学習」(講談社, 2015)
- [Source code for Chainer.optimizers.rmsprop_graves](http://docs.chainer.org/en/stable/_modules/chainer/optimizers/rmsprop_graves.html#RMSpropGraves)
- Alex Graves, Generating Sequences With Recurrent Neural Networks, arXiv:1308.0850 [\[Paper\]](http://arxiv.org/abs/1308.0850)